In [23]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/ubuntu/varios/skforecast


In [35]:
from skforecast.recursive import ForecasterRecursive, ForecasterRecursiveMultiSeries
from skforecast.model_selection import TimeSeriesFold, backtesting_forecaster, backtesting_forecaster_multiseries
from lightgbm import LGBMRegressor

In [25]:
n = 10_000
y = pd.Series(
    np.random.normal(0, 1, n),
    index=pd.date_range('2020-01-01', periods=n, freq='h')
)
y

2020-01-01 00:00:00    1.553241
2020-01-01 01:00:00    0.811703
2020-01-01 02:00:00   -0.894826
2020-01-01 03:00:00   -0.009481
2020-01-01 04:00:00    0.257173
                         ...   
2021-02-20 11:00:00   -0.000411
2021-02-20 12:00:00    1.466122
2021-02-20 13:00:00   -1.535374
2021-02-20 14:00:00    0.833735
2021-02-20 15:00:00    1.146183
Freq: h, Length: 10000, dtype: float64

In [26]:
forecaster = ForecasterRecursive(
    regressor=LGBMRegressor(verbose=-1),
    lags=48
)

In [27]:
%%timeit
forecaster.fit(y=y)

834 ms ± 323 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
%%timeit
forecaster.fit(y=y, store_in_sample_residuals=False)

559 ms ± 23.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
cv = TimeSeriesFold(
    initial_train_size=7_000,
    steps=48,
)

  0%|          | 0/63 [00:00<?, ?it/s]

In [34]:
%%timeit
_ = backtesting_forecaster(
    forecaster=forecaster,
    y=y,
    cv=cv,
    metric='mean_absolute_error',
    verbose=False,
    show_progress=False
)

3.3 s ± 424 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [54]:
n = 1_000
n_series = 500
series = pd.DataFrame(
    np.random.normal(0, 1, n * n_series).reshape(n, n_series),
    index=pd.date_range('2020-01-01', periods=n, freq='h'),
    columns=[f'series_{i}' for i in range(n_series)]
)

In [55]:
forecaster = ForecasterRecursiveMultiSeries(
    regressor = LGBMRegressor(verbose=-1),
    lags     = 48
)

forecaster.fit(series=series, store_in_sample_residuals=False)
print(forecaster.in_sample_residuals_)
forecaster.binner_intervals_

{}


{'series_0': {0: (-0.17827787073514975, -0.03614010053465551),
  1: (-0.03614010053465551, -0.02225029006846725),
  2: (-0.02225029006846725, -0.014093669735334437),
  3: (-0.014093669735334437, -0.0075480650013717504),
  4: (-0.0075480650013717504, -0.0029705320284601597),
  5: (-0.0029705320284601597, 0.0009904296619752497),
  6: (0.0009904296619752497, 0.005706249521471761),
  7: (0.005706249521471761, 0.011921801194319235),
  8: (0.011921801194319235, 0.022606479512717256),
  9: (0.022606479512717256, 0.0917820050519349)},
 'series_1': {0: (-0.2478768652837165, -0.04116077442514718),
  1: (-0.04116077442514718, -0.023920228887087783),
  2: (-0.023920228887087783, -0.014316331198105691),
  3: (-0.014316331198105691, -0.008016873386605757),
  4: (-0.008016873386605757, -0.0028219128391184936),
  5: (-0.0028219128391184936, 0.0016552759085342059),
  6: (0.0016552759085342059, 0.0067853637356419735),
  7: (0.0067853637356419735, 0.013528428872270248),
  8: (0.013528428872270248, 0.0289

In [56]:
%%timeit
forecaster.fit(series=series)

10.1 s ± 471 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [57]:
%%timeit
forecaster.fit(series=series, store_in_sample_residuals=False)

8.84 s ± 27.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
